# Spark SQL & Machine learning
   
   - Beginning 2.0, DataFrame is the new basic structure in spark(built on top of RDD)
   - A dataframe is nothing but a table with columns, rows and headers
   - In this notebook, we will work through a dataset to demostrate spark's SQL-like abilities. We will also look at its machine learning capabilities in context of this dataset.
    
### Dataset
- The chosen dataset is that of Breast Cancer. This set is collected from digitized image of a fine needle aspirate (FNA) of a breast mass. For more information about this dataset, please visit https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
- Each row contains information about a single breast mass along with a diagnosis of malignant/benign for this mass
    
### Problem statement
- We need to learn patterns describing malignant and benign masses and need to be able to place any future samples in either of these buckets.  
- Thus, the problem is a classification one where given a set of probable results, we need to pick one result with confidance. 
    
### Solution
   We will go through the following steps:
   - Read data (Spark SQL)
   - Feature Engineering (Accumulators, Broadcasters, ml/mllib APIs)
   - Data Visualizations (PixieDust, Seaborn)
   - Modeling (Spark ML)
   - Evaluation and prediction (Spark ML)
   - Deployment (Watson ML repository)

In [7]:
#### Machine Learning process ###
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/martinkearn/Content/master/Blogs/Images/MLProcess.PNG")

### Import libraries needed 

#### Copy and run the code below

    from pyspark.sql import functions as F
    from pyspark.sql.types import FloatType, StringType, IntegerType, DoubleType, ArrayType

    from pyspark.mllib.stat import Statistics
    from pyspark.mllib.linalg import Vectors

    from pyspark import AccumulatorParam
    from pyspark import Broadcast

    import seaborn as sns
    import matplotlib.pyplot as plt
    import json
    import pandas as pd

    %matplotlib inline

### Read data from objectStore (can be any source, really)
 Data can be read in any format (as applicable to the native format), most common use cases are reading as json,csv, text or parquet files 
 
 
 Click on the data file and ask to "Insert as SparkSession Table"

### Rename the input variable
    data = df_data_1

### Visualize sample dataset
    data.select(data.columns[:9]).show(10)

### Data Exploration ###

#### Register dataframe as a temp table to query from (write sql on dataframes)

    data.registerTempTable("cancer_data")

### SQL 
- NOTE that the returned object is another Dataframe
- One nice feature of the notebooks and python is that we can show it in a table via Pandas


##### copy and paste code 
    temp_df =  spark.sql("select * from cancer_data where diagnosis=='B' limit 2")

    print ('Type of result:',type(temp_df))
    print ("\n")
    temp_df.toPandas()

### Simple groupby example
   - Remember to perform an action to get your results (sql queries are also transformations :))
    
##### copy and paste code from below

    query = """
    select
        diagnosis ,
        count(*) as diagnosis_count,
        mean(RADIUS_SE)
    from cancer_data
    group by diagnosis 
    order by count(*) desc
    """

    spark.sql(query).toPandas()

## Feature Engineering
   - Dataset = Features + target
   - Transforming features (categorical to numeric, continous to bins, scaling, normalization etc...)
   - Selecting a subset of columns for wide datasets
   - Exploding columns to make additional (synthetic features) for small datasets

#### Feature set is numeric

##### Step 1: Make sure your target is numeric
- "target" is categorical (malignant/benign)
- Our encoding: 1 = malignant(M), 0 = Benign(B) using sql like "when" statement

### review your data

    data.toPandas().head(3)

### Make label numeric
    data = data.withColumnRenamed('diagnosis','label')
    data = data.withColumn('label',F.when(data.label=='M',1).otherwise(0))
    data.toPandas().head(3)

***********************************************************************************

###   Detour  
- Feature engineering is a major task and often involves complicated scripts

What happens when
   - Have custom script based on other datasets/static variables?
   - I want to port over a current script to spark ?

### Accumulators, Broadcasters
- One of the most basic things in scripts are variables (data and metadata). 
- Data vars = DataFrames/RDDs in spark
- Metadata vars = Accumulators/broadcasters 
   
- Metadata vars are not straightforward in the distributed world


##### Accumulators
   - Global variables which can be written into
   
##### Broadcasters
   - Global variables to be read from 
************************************************************************************************


#### Step 2: Select a subset of columns
- Remove columns conveying very little information
   - Zero variance columns
   - Low correlation columns

##### Zero variance test
   - Let us write this test ourselves!

In [4]:
class DictAccumulatorParam(AccumulatorParam):
    def zero(self,initialValue):
        return initialValue

    def addInPlace(self, v1, v2):
        v1.update(v2)
        return v1
    
    
def test_zero_variance(d,acc):
    global df_pd
    col = d[0]
    value = d[1]
    try:
        mean = df_pd.value['mean'][col]
        std = df_pd.value['stddev'][col]
        if float(value)==float(mean) and float(std)==0.0:
            acc.add({col:True})
            return True
        else:
            acc.add({col:False})
            return False
    except KeyError:
        acc.add({col:False})
        return False
    
    
def count_labels(row):
    global benign, malignant
    if row.label==1:
        malignant.add(1)
    else:
        benign.add(1)

#### Broadcast the data variable so that each executor has its own version of it
   - This way, it does need not be shipped to the executor with every call
   - Saves network bandwidth

##### copy and paste code from below
    ### Create a broadcast variable with the summary stats ###
    data = data.withColumn('dummy',F.lit(100))

    ### Correct data types of variables prior to doing statistics ###
    for col in data.columns:
        if col not in ['ID','diagnosis']:
            data = data.withColumn(col,data[col].cast(FloatType()))

    df = data.describe()
    df_pd = df.toPandas()
    df_pd.index = df_pd["summary"]
    del df_pd["summary"]
    df_pd = df_pd.transpose()

    df_pd = sc.broadcast(df_pd)

#### Create and accumulate

##### copy and paste code

    acc = sc.accumulator({}, DictAccumulatorParam())  
    values = data.first().asDict()
    input_value = sc.parallelize(values.items())
    dummy = input_value.map(lambda x:test_zero_variance(x,acc))\
                      .map(lambda x:(x,1))\
                      .reduceByKey(lambda x,y:x+y)
    dummy_collect = dummy.collect()


### view the contents of accumulator
    acc.value

#### Proceed to read the accumulator to make decision to retain/drop columns

### Removing zero variance columns ###

##### copy and run code from below
    for k,v in acc.value.items():
        if v==True:
            data = data.drop(k)

    df = data.toPandas()
    col_corr = sorted(df.corr()['label'].to_dict().items(),key=lambda x:x[1],reverse=True)
    for col in col_corr[-5:]:
        if col[0]!='id':
            data = data.drop(col[0])

- Remove columns with low correlation, such columns contribute to very little information
- Caution: Be wary of multicolinearity ("leaky" columns)

## Lets do some graphs!

###  Here, we look at the columns and their correlations with the target variable.
- We use PixieDust (charting library) for quick charts
- Also takes RDDs as inputs, most of the other charting librabries take pandas dataframes as inputs


##### copy and run code from here
    import pixiedust
    
    df = pd.DataFrame(col_corr)
    df = df.dropna()
    df.columns = ['name','correlation']
    viz_spark_df = sqlContext.createDataFrame(df)

   
    display(viz_spark_df)

### Visualize correlations ####
 - Useful for datasets with small number of columns
 - A heatmap is useful to visualize how variables are related to each other (and not just the target)
 - Very useful while doing NLP applications to visualize similarity between documents (after using TFIDF)
 
 
##### copy and run code from below
    plt.style.use('ggplot')
    values = data.rdd.map(lambda x:list(x.asDict().values()))
    corr_values = Statistics.corr(values)
    names = data.rdd.map(lambda x:list(x.asDict().keys())).first()

    plt.figure(figsize=(10,10))
    sns.set_style("darkgrid")
    sns.heatmap(corr_values,xticklabels=names,yticklabels=names,square=True,vmin=0, vmax=1,
                    cmap="YlGnBu")

## Machine Learning Module layout
##### It divides into two packages:
    - spark.mllib contains the original API built on top of RDDs.
    - spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.
    - Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.


### PCA  - Dimensionality Reduction 
 - Very handy for visualization!

In [9]:
###### PCA Visualize data ######
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import VectorAssembler 

import regex

def g(point,co_ord):
    if co_ord=='x':
        value = regex.split('\s+',point)[0][1:]
    else:
        
        value =  regex.split('\s+',point)[1][:-1]
    return value

feature_cols = list(filter(lambda x:x not in ['id','label','dummy'],data.columns))
assembler = VectorAssembler(inputCols=feature_cols,outputCol='features')
df = assembler.transform(data)

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

pca_result = model.transform(df)
result = pca_result.select("pcaFeatures")

get_x_coord = UserDefinedFunction(lambda point:g(point,'x'), StringType())
get_y_coord = UserDefinedFunction(lambda point:g(point,'y'), StringType())
convert_to_string = UserDefinedFunction(lambda point:str(point.toArray()),StringType())

pca_result = pca_result.withColumn('pcaFeatures_string',convert_to_string(pca_result['pcaFeatures']))
pca_result = pca_result.withColumn('x_coord',get_x_coord(pca_result['pcaFeatures_string']))\
                       .withColumn('y_coord',get_y_coord(pca_result['pcaFeatures_string']))\
                      
pca_result = pca_result.withColumn('x_coord',pca_result.x_coord.cast(FloatType()))\
                       .withColumn('y_coord',pca_result.y_coord.cast(FloatType()))
    
viz_df = pca_result.select('x_coord','y_coord','label').toPandas()
a = sns.lmplot(x='x_coord',y='y_coord',hue='label',data=viz_df,
           fit_reg=False,palette={1:'red',0:'green'},size=6,aspect=1)


### Prepare the dataset for machine learning
    - Split data between test and train (70/30 split)
    - Apply logistic regression and decision tree classifiers
    
##### copy and run code
    data = data.withColumn('label',data['label'].cast(DoubleType()))
    train, test = data.randomSplit([0.7,0.3])
    train.count(), test.count()

#### Build classification models to learn your data 
- Classification models work towards making a Yes/No decision
- Metrics to evaluate the model: Precision/Recall, AUC


In [ ]:
#### Classification metrics ####
def calc_metrics(results):
    metrics = {}
    metrics['tp_0'] = results.filter((results.label==0)&(results.prediction==0)).count()
    metrics['fn_0'] = results.filter((results.label==0)&(results.prediction==1)).count()
    metrics['tn_0'] = results.filter((results.label==1)&(results.prediction==1)).count()
    metrics['fp_0'] = results.filter((results.label==1)&(results.prediction==0)).count()
    
    metrics['tp_1'] = results.filter((results.label==1)&(results.prediction==1)).count()
    metrics['fn_1'] = results.filter((results.label==1)&(results.prediction==0)).count()
    metrics['tn_1'] = results.filter((results.label==0)&(results.prediction==0)).count()
    metrics['fp_1'] = results.filter((results.label==0)&(results.prediction==1)).count()
    
    return metrics
        
### calc precision & recall ###
def precision_recall(results):
    items = calc_metrics(results)
    pre_0 = items['tp_0']/float((items['tp_0']+items['fp_0']))
    pre_1 = items['tp_1']/float((items['tp_1']+items['fp_1']))
    
    recall_0 = items['tp_0']/float((items['tp_0']+items['fn_0']))
    recall_1 = items['tp_1']/float((items['tp_1']+items['fn_1']))
    
    return {'pre_0':pre_0, 'recall_0':recall_0,'pre_1':pre_1,'recall_1':recall_1}

### Pipelines
- Easy to use API
- Define stages to transform your dataset
- Select any estimator (model) for prediction 
- Each pipeline will model a single estimator onto your model

In [ ]:
from pyspark.ml.pipeline import Pipeline

#### Define Assembler to combine feature into a single vetor (feature vector! :) )

#### copy and run code
    from pyspark.ml.linalg import Vectors #linear algebra package, has matrices, arrays, Vectors (dense and sparse)
    from pyspark.ml.feature import VectorAssembler 

    feature_cols = list(filter(lambda x:x not in ['id','label'],data.columns))
    assembler = VectorAssembler(inputCols=feature_cols,outputCol='features')

## Classification Pipelines 
- Logistic Regression
- Tree based classifier

##### Logistic regression #####
    from pyspark.ml.classification import LogisticRegression

    #define estimator and fit data
    estimator = LogisticRegression()
    pipeline = Pipeline(stages=[assembler,estimator])
    lr_model = pipeline.fit(train)

    #get results
    results = lr_model.transform(test)
    precision_recall(results)

In [10]:
### Visualize results from logistic regression ###
pca_subset = pca_result.select('id','features','x_coord','y_coord')
viz_df = pca_subset.join(results,on=['id','features']).select('x_coord','y_coord','prediction','label')
viz_df = viz_df.dropna(how='any',subset=['x_coord','y_coord']).toPandas()

a = sns.lmplot(x='x_coord',y='y_coord',hue='prediction',col='label',data=viz_df,
           fit_reg=False,palette={1:'red',0:'green'},size=6,aspect=1)
a.set_titles(col_template=['Decision Tree results when label=="Benign"','Decision Tree results when label=="Malignant"'])

#### Tree classifier ####
    from pyspark.ml.classification import DecisionTreeClassifier

    # define estimator and fit data
    estimator = DecisionTreeClassifier()
    pipeline = Pipeline(stages=[assembler,estimator])
    tree_model = pipeline.fit(train)

    # get results
    results = tree_model.transform(test)
    precision_recall(results)

In [11]:
### Visualize the results of tree classifier ###
pca_subset = pca_result.select('id','features','x_coord','y_coord')
viz_df = pca_subset.join(results,on=['id','features']).select('x_coord','y_coord','prediction','label')
viz_df = viz_df.dropna(how='any',subset=['x_coord','y_coord']).toPandas()

a = sns.lmplot(x='x_coord',y='y_coord',hue='prediction',col='label',data=viz_df,
           fit_reg=False,palette={1:'red',0:'green'},size=6,aspect=1)

### Clustering Pipeline
- K-means clustering

In [12]:
######## Kmeans clustering, join to the PCA set #########
from numpy import array
from math import sqrt
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=2,featuresCol='features')
model = kmeans.fit(pca_result)
centers = model.clusterCenters()
kmeans_result = model.transform(pca_result).select('id','features','prediction')

viz_df = pca_result.join(kmeans_result,on=['id','features'],how='inner')\
                   .select('id','features','x_coord','y_coord','prediction','label')

viz_df = viz_df.dropna(how='any',subset=['x_coord','y_coord']).toPandas()

sns.lmplot(x='x_coord',y='y_coord',hue='prediction',col='label',data=viz_df,
           fit_reg=False,palette={1:'red',0:'green'},size=6,aspect=1)

### Deploy to Cloud by using Watson Machine Learning Repo
   Please follow demonstration! :D
